## Experimenting with the library

In [ ]:
import os 

In [ ]:
os.chdir("../")

In [ ]:
print(os.getcwd())
musicPath = os.getcwd()

In [ ]:
os.chdir(musicPath+"/code/midicsv/")

In [ ]:
import subprocess as sp

In [ ]:
outputFile =open(musicPath+'/output/test.csv','w+')
errorFile =open('test.csv','w+')

In [ ]:
print(os.getcwd())
os.listdir()

In [ ]:
sp.call(['./midicsv', '-v', musicPath+"/data/maestro/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi"], stdout=outputFile, stderr=errorFile)
outputFile.close()
errorFile.close()

## Feed into Pandas DF

In [3]:
import os 
import subprocess
import sys
import pandas as pd
import math
os.chdir("../")
musicPath = os.getcwd()
os.chdir(musicPath+"/code/midicsv/")

In [4]:
def midiToCsv(restofPath):
    cmd = ['./midicsv', '-v', musicPath+restofPath]
    a = subprocess.Popen(cmd, stdout=subprocess.PIPE)

    if sys.version_info[0] < 3: 
        from StringIO import StringIO
    else:
        from io import StringIO

    b = StringIO(a.communicate()[0].decode('utf-8'))
    df = pd.read_csv(b,names=["track","time", "type", "channel","note","velocity", "other", "encoded"], sep=",")
    return df

In [5]:
def getQuarterNoteRate(df):
    noteRate = df['velocity'].iloc[0]
    return noteRate

In [6]:
def setQuarterNoteTime(df, noteRate, sampleRate):
    df['time']=df['time'].apply(lambda x:int(math.floor(x/(noteRate/sampleRate))))
    return df

In [7]:
def removeControlC(df):
    indexNames = df[ df['type'].str.contains('Control_c')].index
    # Delete these row indexes from dataFrame
    out = df.drop(indexNames )
    #print(out)
    return out

In [8]:
df = midiToCsv("/data/maestro/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi")
noteRate = getQuarterNoteRate(df)
withoutC = removeControlC(df)
diffTime = setQuarterNoteTime(withoutC, noteRate, 12)
diffTime.head(20)

,track,time,type,channel,note,velocity,other,encoded
0,0,0,Header,1.0,2.0,480.0,NaN,NaN
1,1,0,Start_track,NaN,NaN,NaN,NaN,NaN
2,1,0,Tempo,500000.0,NaN,NaN,NaN,NaN
3,1,0,Time_signature,4.0,2.0,24.0,8.0,NaN
4,1,0,End_track,NaN,NaN,NaN,NaN,NaN
5,2,0,Start_track,NaN,NaN,NaN,NaN,NaN
6,2,0,Program_c,0.0,0.0,NaN,NaN,NaN
13,2,26,Note_on_c,0.0,71.0,60.0,NaN,NaN
16,2,28,Note_on_c,0.0,71.0,0.0,NaN,NaN
20,2,30,Note_on_c,0.0,55.0,44.0,NaN,NaN


In [42]:
def getNote(x):
    arr = x.split('|')
    return arr[1]

def makeString(df):
    df['note'] = df['note'].fillna(0).astype(int)
    df['encoded'] = df['type']+"|"+df['note'].astype(str)+"|"+df['velocity'].astype(str)
    df['finalEncode'] = df['encoded'].apply(lambda x: "off" + getNote(x)  if x.endswith("|0.0") else ("on" + getNote(x)))

    return df


In [10]:
diffTime['note']

0         2.0
1         NaN
2         NaN
3         2.0
4         NaN
         ... 
29502    67.0
29503    59.0
29504    62.0
29520     NaN
29521     NaN
Name: note, Length: 15797, dtype: float64

In [43]:
makeString(diffTime).head(20)

,track,time,type,channel,note,velocity,other,encoded,finalEncode
0,0,0,Header,1.0,2,480.0,NaN,Header|2|480.0,on2
1,1,0,Start_track,NaN,0,NaN,NaN,Start_track|0|nan,on0
2,1,0,Tempo,500000.0,0,NaN,NaN,Tempo|0|nan,on0
3,1,0,Time_signature,4.0,2,24.0,8.0,Time_signature|2|24.0,on2
4,1,0,End_track,NaN,0,NaN,NaN,End_track|0|nan,on0
5,2,0,Start_track,NaN,0,NaN,NaN,Start_track|0|nan,on0
6,2,0,Program_c,0.0,0,NaN,NaN,Program_c|0|nan,on0
13,2,26,Note_on_c,0.0,71,60.0,NaN,Note_on_c|71|60.0,on71
16,2,28,Note_on_c,0.0,71,0.0,NaN,Note_on_c|71|0.0,off71
20,2,30,Note_on_c,0.0,55,44.0,NaN,Note_on_c|55|44.0,on55


In [ ]:
diffTime['encoded'].head(20)